In [28]:
import pandas as pd
import csv
import pickle
import numpy as np
from IPython.display import display
import multiprocessing
from multiprocessing import Pool
import math
from tqdm import tqdm
import time
import traceback
from sklearn.externals import joblib
import xgboost as xgb
import json
import time

In [29]:
path = "/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/"
model = joblib.load(path+"/xgb_model_GIVENCHY (2).dat")
#model = joblib.load(path+"xgb_model_14_hrs.dat")

In [30]:
def load_pickle_file(pickled_file):
    print(f'Loading data file from {pickled_file}')
    infile = open(pickled_file,'rb')
    unpickled_file = pickle.load(infile)
    print(f'Loaded {len(unpickled_file)} entries')
    infile.close()
    return unpickled_file
          
    
def save_pickle_file(path, data):
    print('Dumping data to path {}'.format(path))
    with open(path, 'wb') as file:
        pickle.dump(data, file)
    print('Finished dumping data to path {}'.format(path))


def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)


def safe_division(x, y):
    if y == 0:
        return 0
    else:
        return x/y



In [31]:
initial_features = load_pickle_file(path+'keynode_initial_features.pkl')
initial_dataset = load_pickle_file(path+'network_simulation_keynode_initial.pkl')
users = load_pickle_file(path+"80000users_3.pkl")
users.reset_index(drop =True , inplace =True)


Loading data file from /Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/keynode_initial_features.pkl
Loaded 70013 entries
Loading data file from /Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/network_simulation_keynode_initial.pkl
Loaded 70013 entries
Loading data file from /Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/80000users_3.pkl
Loaded 70013 entries


In [32]:
betweenness_centrality= pd.read_csv(path+'70013users_betweenness_centrality.csv')
sorted_df = betweenness_centrality.sort_values(by = ['betweenness_centrality'],ascending = False)
seeds= sorted_df[0:10]

In [33]:
seeds

,created_at,user_favourites_count,followers_count,friends_count,user_id,user_listed_count,user_statuses_count,source_candidates,target_nodes,user_created_days,normalized_user_statuses_count,normalized_user_followers_count,normalized_user_favourites_count,normalized_user_listed_count,normalized_user_friends_count,avg_neighbour_degree,betweenness_centrality,avg_neighbour_betweenness
1441,2007-03-05 22:08:25,11,101454968,620510,813286,228509,15517,"[1415, 1485, 1434, 1426, 1535, 1460, 1459, 153...","[13763552, 14957147, 5988062, 14647570, 462045...",4025,3.855155,25206.203230,0.002733,56.772422,154.163975,33.151899,0.131855,0.000468
2242,2008-10-31 22:26:54,22503,7966561,1517036,17093617,47381,62261,"[2241, 2261, 2220, 2277, 2270, 2297, 2319, 225...","[5380672, 9726632, 87812442, 16420518, 1662418...",3419,18.210295,2330.085113,6.581749,13.858146,443.707517,38.279683,0.056314,0.000600
10261,2008-08-14 03:50:42,959,75965729,35685,15846407,107500,17032,"[10143, 10123, 10128, 10199, 10233, 10192, 102...","[813286, 7259302, 16157855, 65691824, 16031927...",3498,4.869068,21716.903659,0.274157,30.731847,10.201544,39.471910,0.050202,0.001164
806,2007-04-03 20:44:17,2170,503906,27988,3359851,4589,17668,"[797, 834, 800, 819, 848, 822, 793, 829, 820, ...","[16265412, 50393960, 41494136, 20567578, 19663...",3996,4.421421,126.102603,0.543043,1.148398,7.004004,49.927771,0.035416,0.000624
451,2008-07-15 11:45:30,626,12754295,50151,15439395,56685,22992,"[479, 510, 482, 541, 493, 495, 507, 580, 536, ...","[14824849, 6572452, 14424667, 7081402, 1183041...",3528,6.517007,3615.162982,0.177438,16.067177,14.215136,37.545455,0.026164,0.001486
1635,2009-03-03 21:05:23,1419,1371396,364,22679419,9761,41886,"[1634, 1610, 1645, 1652, 507, 1699, 1672, 1676...","[15234657, 146930781, 10886182, 16252002, 3359...",3296,12.708131,416.078883,0.430522,2.961468,0.110437,62.560241,0.022146,0.000474
155,2007-03-26 22:30:49,18227,15439864,32116,2367911,30911,262416,"[67, 78, 131, 83, 182, 52, 119, 96, 44, 168, 1...","[314316607, 69592091, 18996905, 16870853, 1762...",4004,65.538462,3856.109890,4.552198,7.720030,8.020979,40.725664,0.018843,0.001550
2077,2007-03-12 01:28:01,865,9840512,2780,972651,128856,303735,"[2076, 2058, 2114, 2158, 2149, 2119, 2123, 216...","[15901190, 899701, 657863, 115485051, 12925072...",4019,75.574770,2448.497636,0.215228,32.061707,0.691714,52.262887,0.017301,0.001731
2244,2008-01-23 20:36:46,8029,1045578,229371,12611642,17857,94358,"[2217, 2250, 2213, 2224, 2329, 1685, 2272, 232...","[123879425, 247565758, 756278, 28133097, 11404...",3701,25.495272,282.512294,2.169414,4.824912,61.975412,65.955645,0.017128,0.000538
2758,2009-07-24 15:08:06,164,1078476,293462,59804598,5376,2993295,"[2757, 2781, 2722, 2833, 2754, 2839, 2717, 277...","[16624756, 27707960, 41209552, 13434622, 24593...",3153,949.348240,342.047574,0.052014,1.705043,93.073898,48.891277,0.015266,0.000518


In [34]:
in_degree = list(initial_dataset.friends_count)
out_degree = list(initial_dataset.followers_count)
degree = in_degree + out_degree

In [35]:
def seed_initialization():
    seed_user_ids = list(seeds['user_id'])
    for seed_id in seed_user_ids:
        initial_features.loc[initial_features['user_id'] == seed_id,'infected_status'] = True
        initial_features.loc[initial_features['user_id'] == seed_id,'infection_time'] = 0
        initial_features.loc[initial_features['user_id'] == seed_id,'infected_status'] = True
        index = initial_features.index[initial_features['user_id'] == seed_id]
        initial_features.loc[initial_features['user_id'] == seed_id,'seed_index'] = index
        initial_features.loc[initial_features['user_id'] == seed_id,'generation'] = 0
        initial_features.loc[initial_features['user_id'] == seed_id,'time_lapsed'] = 0
        
        index = initial_dataset.index[initial_dataset['id'] == seed_id]
        initial_dataset.loc[initial_dataset['id'] == seed_id,'seed_index'] = index
        initial_dataset.loc[initial_dataset['id'] == seed_id,'generation'] = 0
        initial_dataset.loc[initial_dataset['id'] == seed_id,'time_lapsed'] = 0
        initial_dataset.loc[initial_dataset['id'] == seed_id,'time_since_seed'] = 0
        
    

In [36]:
def process_data(source_id,target_id,features,network_simulation,current_time):
    
    if isinstance(network_simulation.loc[target_id,'exposed_source_candidates'],list):
        
        source_candidates = sorted(network_simulation.loc[target_id,'exposed_source_candidates'])
        sources = source_candidates

        first_source_index = source_candidates[0]
        first_source_row = network_simulation.loc[first_source_index]
        first_source_seed_row = network_simulation.loc[first_source_row['seed_index']]

        sources_dataframe = network_simulation.loc[sources]
        degreeList = list(degree[i] for i in sources)
        inDegreeList = list(in_degree[i] for i in sources)
        outDegreeList = list(out_degree[i] for i in sources)
        degreeList = list(network_simulation.loc[i, 'followers_count'] + network_simulation.loc[i, 'friends_count']  for i in sources)
        timeList = [current_time - network_simulation.loc[x,'time_lapsed'] for x in sources]


        last_source_index = sources[-1]    
        try:
            last_source_row = network_simulation.loc[last_source_index]
            last_source_seed_row = network_simulation.loc[last_source_row['seed_index']]
        except:
            print(f"target_index:{target_id}")
            print(f"last_source_index:{last_source_index}")
            

        #Extraction
        #Columns which are added for simulation, but they are not used as features for model prediction

        user_row = network_simulation.loc[target_id]


        # UsM: User metadata                    

        features.loc[target_id,'UsM_deltaDays0'] = first_source_row.user_created_days
        features.loc[target_id,'UsM_statusesCount0'] = first_source_row.statuses_count
        features.loc[target_id,'UsM_followersCount0'] = first_source_row.followers_count
        features.loc[target_id,'UsM_favouritesCount0'] = first_source_row.favourites_count
        features.loc[target_id,'UsM_friendsCount0'] = first_source_row.friends_count
        features.loc[target_id,'UsM_listedCount0'] = first_source_row.listed_count
        features.loc[target_id,'UsM_normalizedUserStatusesCount0'] = first_source_row.normalized_statuses_count
        features.loc[target_id,'UsM_normalizedUserFollowersCount0'] = first_source_row.normalized_followers_count
        features.loc[target_id,'UsM_normalizedUserFavouritesCount0'] = first_source_row.normalized_favourites_count
        features.loc[target_id,'UsM_normalizedUserListedCount0'] = first_source_row.normalized_listed_count
        features.loc[target_id,'UsM_normalizedUserFriendsCount0'] = first_source_row.normalized_friends_count
        features.loc[target_id,'UsM_deltaDays-1'] = last_source_row.user_created_days
        features.loc[target_id,'UsM_statusesCount-1'] = last_source_row.statuses_count
        features.loc[target_id,'UsM_followersCount-1'] = last_source_row.followers_count
        features.loc[target_id,'UsM_favouritesCount-1'] = last_source_row.favourites_count
        features.loc[target_id,'UsM_friendsCount-1'] = last_source_row.friends_count
        features.loc[target_id,'UsM_listedCount-1'] = last_source_row.listed_count
        features.loc[target_id,'UsM_normalizedUserStatusesCount-1'] = last_source_row.normalized_statuses_count
        features.loc[target_id,'UsM_normalizedUserFollowersCount-1'] = last_source_row.normalized_followers_count
        features.loc[target_id,'UsM_normalizedUserFavouritesCount-1'] = last_source_row.normalized_favourites_count
        features.loc[target_id,'UsM_normalizedUserListedCount-1'] = last_source_row.normalized_listed_count
        features.loc[target_id,'UsM_normalizedUserFriendsCount-1'] = last_source_row.normalized_friends_count
        # TwM: Tweet metadata
        features.loc[target_id,'TwM_t0'] = round(timeList[0], 1)
        features.loc[target_id,'TwM_tSeed0'] = round(current_time - first_source_seed_row['time_lapsed'], 1)
        features.loc[target_id,'TwM_t-1'] = round(timeList[-1], 1)
        features.loc[target_id,'TwM_tSeed-1'] = round(current_time - last_source_seed_row['time_lapsed'], 1)
        features.loc[target_id,'TwM_tCurrent'] = current_time
        # Nw: Network
        features.loc[target_id,'Nw_degree'] = degree[target_id]
        features.loc[target_id,'Nw_inDegree'] = in_degree[target_id]
        features.loc[target_id,'Nw_outDegree'] = out_degree[target_id]
        features.loc[target_id,'Nw_degree0'] = degree[first_source_index]
        features.loc[target_id,'Nw_inDegree0'] = in_degree[first_source_index]
        features.loc[target_id,'Nw_outDegree0'] = out_degree[first_source_index]
        features.loc[target_id,'Nw_degree-1'] = degree[last_source_index]
        features.loc[target_id,'Nw_inDegree-1'] = in_degree[last_source_index]
        features.loc[target_id,'Nw_outDegree-1'] = out_degree[last_source_index]
        features.loc[target_id,'Nw_degreeSeed0'] = degree[int(first_source_row['seed_index'])]
        features.loc[target_id,'Nw_inDegreeSeed0'] = in_degree[int(first_source_row['seed_index'])]
        features.loc[target_id,'Nw_outDegreeSeed0'] = out_degree[int(first_source_row['seed_index'])]
        features.loc[target_id,'Nw_degreeSeed-1'] = degree[int(last_source_row['seed_index'])]
        features.loc[target_id,'Nw_inDegreeSeed-1'] = in_degree[int(last_source_row['seed_index'])]
        features.loc[target_id,'Nw_outDegreeSeed-1'] = out_degree[int(last_source_row['seed_index'])]
        # SNw: Spreading Network
        features.loc[target_id,'SNw_nFriendsInfected'] = len(sources)
        features.loc[target_id,'SNw_friendsInfectedRatio'] = safe_division(len(sources), user_row['friends_count'])
        features.loc[target_id,'SNw_generation0'] = first_source_row['generation']
        features.loc[target_id,'SNw_generation-1'] = last_source_row['generation']
        features.loc[target_id,'SNw_timeSinceSeed0'] = first_source_row['time_since_seed']
        features.loc[target_id,'SNw_timeSinceSeed-1'] = last_source_row['time_since_seed']

        infected_dataframe = network_simulation[network_simulation.time_lapsed <= current_time]
        total_nodes_infected = infected_dataframe.shape[0]
        total_in_degree = sum(infected_dataframe.friends_count)
        total_out_degree = sum(infected_dataframe.followers_count)

        features.loc[target_id,'SNw_totalNodesInfected'] = total_nodes_infected
        features.loc[target_id,'SNw_nodeInfectedCentrality'] = len(sources)/total_nodes_infected
        features.loc[target_id,'SNw_totalInDegree'] = total_in_degree
        features.loc[target_id,'SNw_totalOutDegree'] = total_out_degree
        features.loc[target_id,'SNw_inDegreeCentrality'] = in_degree[target_id]/total_in_degree
        features.loc[target_id,'SNw_inDegreeCentrality0'] = in_degree[first_source_index]/total_in_degree
        features.loc[target_id,'SNw_inDegreeCentrality-1'] = in_degree[last_source_index]/total_in_degree
        features.loc[target_id,'SNw_outDegreeCentrality'] = out_degree[target_id]/total_out_degree
        features.loc[target_id,'SNw_outDegreeCentrality0'] = out_degree[first_source_index]/total_out_degree
        features.loc[target_id,'SNw_outDegreeCentrality-1'] = out_degree[last_source_index]/total_out_degree
        features.loc[target_id,'SNw_inDegreeCentralitySeed0'] = in_degree[int(first_source_row['seed_index'])]/total_in_degree
        features.loc[target_id,'SNw_outDegreeCentralitySeed0'] = out_degree[int(first_source_row['seed_index'])]/total_out_degree
        features.loc[target_id,'SNw_inDegreeCentralitySeed-1'] = in_degree[int(last_source_row['seed_index'])]/total_in_degree
        features.loc[target_id,'SNw_outDegreeCentralitySeed-1'] = out_degree[int(last_source_row['seed_index'])]/total_out_degree
        # Stat: Statistical
        features.loc[target_id,'Stat_average_kOut'] = round(mean(degreeList), 1)
        features.loc[target_id,'Stat_average_t'] = round(mean(timeList), 1)
        features.loc[target_id,'Stat_average_deltaDays'] = sources_dataframe.user_created_days.mean()
        features.loc[target_id,'Stat_average_statusesCount'] = sources_dataframe.statuses_count.mean()
        features.loc[target_id,'Stat_average_followersCount'] = sources_dataframe.followers_count.mean()
        features.loc[target_id,'Stat_average_favouritesCount'] = sources_dataframe.favourites_count.mean()
        features.loc[target_id,'Stat_average_friendsCount'] = sources_dataframe.friends_count.mean()
        features.loc[target_id,'Stat_average_listedCount'] = sources_dataframe.listed_count.mean()
        features.loc[target_id,'Stat_average_normalizedUserStatusesCount'] = sources_dataframe.normalized_statuses_count.mean()
        features.loc[target_id,'Stat_average_normalizedUserFollowersCount'] = sources_dataframe.normalized_followers_count.mean()
        features.loc[target_id,'Stat_average_normalizedUserFavouritesCount'] = sources_dataframe.normalized_favourites_count.mean()
        features.loc[target_id,'Stat_average_normalizedUserListedCount'] = sources_dataframe.normalized_listed_count.mean()
        features.loc[target_id,'Stat_average_normalizedUserFriendsCount'] = sources_dataframe.normalized_friends_count.mean()
        features.loc[target_id,'Stat_max_kOut'] = max(degreeList)
        features.loc[target_id,'Stat_min_kOut'] = min(degreeList)
    #processed_dataframe = pd.DataFrame(features)
    return features



In [37]:
def simulation(features,dataset,network_simulation,current_time):
    infected_users_indices = network_simulation[network_simulation['time_lapsed'].isnull() == False].index.values
    for i in infected_users_indices:
        if isinstance(network_simulation.loc[i,'followers_list'],list):            
            followers_indices = []
            for x in network_simulation.loc[i].followers_list:
                followers_indices.append(network_simulation[network_simulation['id'] == x].index.values.item())
            
            uninfected_followers_indices = [y for y in followers_indices if np.isnan(network_simulation.loc[y, 'time_lapsed']) == True]

            if len(uninfected_followers_indices) > 0:
                for j in uninfected_followers_indices:
                    source_index = i
                    target_index = j
                    processed_dataframe = process_data(source_index,target_index,features,network_simulation,current_time)                    
                    valid_row = processed_dataframe.iloc[[target_index]]
                    valid = valid_row.drop(['user_id','infected_status','infection_time','followers_list',
                                            'Nw_inDegree', 'generation', 'time_lapsed', 'seed_index', 'Nw_outDegree'],axis=1)
                    
                    valid = valid.astype('float64')                    

                    columns = list(valid.columns)
                    X = valid[columns]
                    
                    pre_data = xgb.DMatrix(X)
                    infec = model.predict(pre_data)
                    if infec > 0.75:
                        print("Infected")
                        network_simulation.loc[target_index,'time_lapsed'] = current_time
                        network_simulation.loc[target_index,'source_index'] = source_index
                        if np.isnan(network_simulation.loc[target_index,'seed_index']):
                            network_simulation.loc[target_index,'seed_index'] = source_index
                        network_simulation.loc[target_index,'generation'] = network_simulation.loc[source_index,'generation'] + 1
                        seed_index = network_simulation.loc[target_index,'seed_index']
                        network_simulation.loc[target_index,'time_since_seed'] = current_time - network_simulation.loc[seed_index,'time_lapsed']
                        followers_of_node = network_simulation.loc[target_index,'followers_list'] 
                        if isinstance(followers_of_node,list):
                            for f in followers_of_node:
                                if np.isnan(network_simulation[network_simulation['id'] == f]['time_lapsed'].values[0]):
                                    follower_index = network_simulation[network_simulation['id'] == f].index.values
                                    try:
                                        list(network_simulation.loc[follower_index,'exposed_source_candidates'].values).append(f)
                                    except:
                                        print(f"exposed_source_candidates:{network_simulation.loc[follower_index,'exposed_source_candidates'].values}")
                                        print(f"f:{f}")

    return network_simulation

In [38]:
total_time_duration = 8*60
interval = 30
#current_time = 360
#current_time = 420
#current_time = 480
#current_time = 540
#current_time = 720
current_time = 30


seed_initialization()
features = initial_features
network_simulation = initial_dataset

print("Simulation started")
start_time = time.time()

while current_time < total_time_duration:
    print(f"current_time:{current_time}")
    network_simulation = simulation(features,initial_dataset,network_simulation,current_time)
    #print(f"current_time:{current_time}")
    current_time += interval
    
print(f"Simulation finished after {round((time.time() - start_time)/60,2)} minutes")
    


Simulation started
current_time:30
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
In

Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
I

Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
I

Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
I

Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
I

...

Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
I

Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
I

Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
I

Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
I

Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
current_time:270
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
Infected
In

In [39]:
network_simulation.to_csv(path+'simulation_result_key_node_betweenness_0_10_10_start_time_30_0point75.csv')

In [42]:
network_simulation.groupby(['generation']).count()['id']

generation
0        10
1      3848
2     20473
3     18123
4      7281
5      2580
6       959
7       414
8       228
9       132
10       25
11        7
12        2
13        1
Name: id, dtype: int64

In [41]:
network_simulation.groupby(['time_lapsed']).count()['id']

time_lapsed
0         10
30      3848
60     20473
90     18123
120     7281
150     2580
180      959
210      414
240      228
270      132
300       25
330        7
360        2
390        1
Name: id, dtype: int64

In [43]:
df = pd.read_csv(path+'simulation_result_key_node_avg_neighbour_betweenness_0_10_10_start_from_30.csv')

In [44]:
df.groupby(['time_lapsed']).count()['id']

time_lapsed
0.0         10
30.0         1
60.0       714
90.0      8815
120.0    22418
150.0    14112
180.0     6565
210.0     2482
240.0      984
270.0      445
300.0      214
330.0       78
360.0       46
390.0       11
420.0        3
Name: id, dtype: int64

In [49]:
network_simulation.groupby(['generation']).count()['id'].to_csv(path+'simulation_result_key_node_degree_top_10_generation_count.csv')

In [40]:
len(network_simulation[network_simulation['time_lapsed'].isnull() == False])

54083

In [ ]:
#Precision Recall:

simulation_comparison = pd.concat([network_simulation['time_lapsed'],users['time_lapsed']],axis = 1)
column_names = ['nw_time_lapsed','usr_time_lapsed']
simulation_comparison.columns = column_names

simulation_comparison['both_infected'] = simulation_comparison.apply(lambda x: 1 if (np.isnan(x['nw_time_lapsed']) == False)
                                                                          & (np.isnan(x['usr_time_lapsed']) == False)
                                                                     else 0,axis = 1)

true_positive = simulation_comparison['both_infected'].sum()
total_predicted_positive = len(simulation_comparison[np.isnan(simulation_comparison['nw_time_lapsed']) == False])
precision = true_positive/total_predicted_positive
total_actual_positive = len(simulation_comparison[np.isnan(simulation_comparison['usr_time_lapsed']) == False])
recall = true_positive/total_actual_positive
f1_score = (2*(precision * recall))/(precision + recall)

# accuracy['simulation_start_time'].append(6)
# accuracy['precision'].append(precision)
# accuracy['recall'].append(recall)

# accuracy['f1_score'].append(f1_score)






In [ ]:
accuracy = {
    
    'simulation_start_time':[],
    'precision':[],
    'recall':[],
    'f1_score':[]
    }

accuracy['simulation_start_time'].append(14)
accuracy['precision'].append(precision)
accuracy['recall'].append(recall)

accuracy['f1_score'].append(f1_score)
result = pd.DataFrame(accuracy)
result.to_csv(path+'simulation_accuracy_14hrs_14hrs_model.csv')

In [ ]:
true_positive = simulation_comparison['both_infected'].sum()

In [ ]:
total_predicted_positive = len(simulation_comparison[np.isnan(simulation_comparison['nw_time_lapsed']) == False])

In [ ]:
precision = true_positive/total_predicted_positive

In [ ]:
total_actual_positive = len(simulation_comparison[np.isnan(simulation_comparison['usr_time_lapsed']) == False])

In [ ]:
recall = true_positive/total_actual_positive

In [ ]:
f1_score = (2*(precision * recall))/(precision + recall)

In [ ]:
precision

In [ ]:
recall

In [ ]:
f1_score

In [ ]:
total_predicted_positive

In [ ]:
total_actual_positive

In [ ]:
true_positive

In [12]:
network_simulation['source_candidates']

0       [3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...
1                                                     [0]
2       [0, 12, 21, 32, 48, 748, 819, 972, 2358, 2778,...
3                                           [0, 115, 577]
4                                                     NaN
5       [0, 20, 577, 1012, 1591, 1656, 1807, 1953, 320...
6                                [0, 21, 748, 1912, 1929]
7                                                     [0]
8                                                     [0]
9                                                     [0]
10                                                    [0]
11                                                    [0]
12                                       [0, 2, 32, 1347]
13                                                    [0]
14      [0, 1, 16, 306, 577, 845, 879, 1012, 1084, 159...
15      [0, 16, 577, 1012, 1084, 1953, 2187, 2467, 320...
16      [0, 14, 15, 21, 108, 748, 879, 1008, 1084, 159...
17            

In [41]:
i = 1723
[network_simulation[network_simulation['id'] == x].index.values.item() for x in network_simulation.loc[i].followers_list]



ValueError: can only convert an array of size 1 to a Python scalar

In [39]:
seeds

,Unnamed: 0,created_at,user_favourites_count,followers_count,friends_count,user_id,user_listed_count,user_statuses_count,source_candidates,infec_candidates,user_created_days,normalized_user_statuses_count,normalized_user_followers_count,normalized_user_favourites_count,normalized_user_listed_count,normalized_user_friends_count
0,1723,2007-11-26 19:50:22,2875,650,181,10632552,42,7513,[525],[806],3759,1.998670,0.172918,0.764831,0.011173,0.048151
1,22956,2009-07-13 03:45:06,0,2927,3788,56278217,46,24843,"[2758, 5800]",[806],3165,7.849289,0.924803,0.000000,0.014534,1.196840
2,31539,2009-09-16 06:09:47,0,18,27,74666155,0,137,[],[806],3100,0.044194,0.005806,0.000000,0.000000,0.008710
3,33519,2009-07-19 21:10:08,0,7,9,58294702,1,128,"[33518, 33534, 33472]",[806],3158,0.040532,0.002217,0.000000,0.000317,0.002850
4,35831,2011-05-02 12:15:12,0,6,35,291662107,0,235,[35830],[806],2507,0.093738,0.002393,0.000000,0.000000,0.013961


In [40]:
network_simulation[network_simulation['time_lapsed'].isnull() == False].index.values

array([ 1723, 22956, 31539, 33519, 35831])

In [42]:
network_simulation.loc[1723].followers_list

[806]

In [43]:
for x in network_simulation.loc[1723].followers_list:
    network_simulation[network_simulation['id'] == x].index.values.item() 

ValueError: can only convert an array of size 1 to a Python scalar

In [81]:
network_simulation[network_simulation['id'] == 3359851]

,id,time_lapsed,favourites_count,followers_count,friends_count,listed_count,statuses_count,exposed_source_candidates,source_index,seed_index,generation,time_since_seed,user_created_days,normalized_statuses_count,normalized_followers_count,normalized_favourites_count,normalized_listed_count,normalized_friends_count,friends_list,followers_list
806,3359851,NaN,2170,503906,27988,4589,17668,NaN,NaN,NaN,NaN,NaN,3996,4.421421,126.102603,0.543043,1.148398,7.004004,"[797, 834, 800, 819, 848, 822, 793, 829, 820, ...","[16265412, 50393960, 41494136, 20567578, 19663..."


In [48]:
initial_dataset[initial_dataset['id'] == 806]

,id,time_lapsed,favourites_count,followers_count,friends_count,listed_count,statuses_count,exposed_source_candidates,source_index,seed_index,...,user_created_days,normalized_statuses_count,normalized_followers_count,normalized_favourites_count,normalized_listed_count,normalized_friends_count,source_nodes,target_nodes,friends_list,followers_list


In [79]:
network_simulation[network_simulation['time_lapsed'].isnull() == False]

,id,time_lapsed,favourites_count,followers_count,friends_count,listed_count,statuses_count,exposed_source_candidates,source_index,seed_index,generation,time_since_seed,user_created_days,normalized_statuses_count,normalized_followers_count,normalized_favourites_count,normalized_listed_count,normalized_friends_count,friends_list,followers_list
9090,23867268,0,421,281,665,10,9669,NaN,NaN,9090,0,0,3288,2.940693,0.085462,0.128041,0.003041,0.202251,[],"[42361118, 3359851]"
21931,216829577,0,0,7,20,0,65,NaN,NaN,21931,0,0,2672,0.024326,0.002620,0.000000,0.000000,0.007485,[],"[59804598, 3359851]"
24305,386428938,0,20,641,165,12,456,NaN,NaN,24305,0,0,2349,0.194125,0.272882,0.008514,0.005109,0.070243,"[24332, 525]","[59804598, 5442012]"
43897,24446042,0,0,74388,18684,4137,6655,NaN,NaN,43897,0,0,3285,2.025875,22.644749,0.000000,1.259361,5.687671,"[43877, 1124, 43876, 1106, 1056, 43862, 43868,...",[5442012]
50718,80873238,0,3,102,54,0,259,NaN,NaN,50718,0,0,3077,0.084173,0.033149,0.000975,0.000000,0.017550,"[3409, 3388, 9016]",[18581803]
59227,20039736,0,3669,5216,978,146,88227,NaN,NaN,59227,0,0,3324,26.542419,1.569194,1.103791,0.043923,0.294224,[1238],[59804598]
61255,82146778,0,240,23143,34,589,530,NaN,NaN,61255,0,0,3072,0.172526,7.533529,0.078125,0.191732,0.011068,"[13385, 61266, 61246, 3325, 61242, 1275, 13142...",[22679419]
63042,35604056,0,0,9,22,0,0,NaN,NaN,63042,0,0,3242,0.000000,0.002776,0.000000,0.000000,0.006786,[],[3359851]
65942,21614972,0,55,112,253,8,466,NaN,NaN,65942,0,0,3305,0.140998,0.033888,0.016641,0.002421,0.076551,[],[15846407]
66309,23912766,0,2316,185972,2567,688,6088,NaN,NaN,66309,0,0,3288,1.851582,56.560827,0.704380,0.209246,0.780718,"[66249, 66250, 66243, 6327]",[813286]


In [83]:
network_simulation.loc[9090].followers_list

[42361118, 3359851]

In [84]:
i

1723

In [87]:
network_simulation[network_simulation['id'] == 3359851].index.values.item()

806

In [5]:
df = pd.read_csv(path+'simulation_result_key_node_degree_top_10.csv')

In [6]:
len(df[df['time_lapsed'].isnull() == False])

56458

In [7]:
df.groupby(['generation']).count()['id']

generation
0.0        10
1.0      4548
2.0     20230
3.0     18885
4.0      8146
5.0      2799
6.0       996
7.0       422
8.0       245
9.0       125
10.0       46
11.0        3
12.0        2
13.0        1
Name: id, dtype: int64